# Imports

In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
import seaborn as sns
import numpy as np
import datetime as dt
from collections import Counter
import requests
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
from shapely.geometry import LineString, Point
from shapely import wkt
from dotenv import load_dotenv, find_dotenv
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
from folium.vector_layers import Circle, CircleMarker
import matplotlib
import ast

# Data

In [2]:
# import renatl info and drop irrelevant columns
bike_df_all = pd.read_csv('raw_data/bike_loans.csv', sep=';').drop(columns=['Importe', 
                                                                        'DescripcionImporte', 
                                                                        'operario'])
bike_df_all['Fecha_Prestamo'] = bike_df_all['Fecha_Prestamo'].astype('datetime64[ns]')
bike_df_all['Fecha_Devolucion'] = bike_df_all['Fecha_Devolucion'].astype('datetime64[ns]')

bike_df_all.dtypes

Id_Historico_Prestamo               int64
Id_Usuario                          int64
Id_Tag_Bicicleta                   object
Fecha_Prestamo             datetime64[ns]
Fecha_Devolucion           datetime64[ns]
Id_Aparcamiento_Origen              int64
Posicion_Origen                     int64
Id_Aparcamiento_Destino           float64
Posicion_Destino                  float64
Num_Bici_Hist                     float64
dtype: object

In [3]:
bike_df_all = bike_df_all.dropna()

# Compute bike count per station per hour

In [4]:
# filter to 2019 data => most representative year
bikes_19 = bike_df_all[bike_df_all.Fecha_Devolucion.dt.year == 2019].reset_index()
bikes_19['Id_Aparcamiento_Destino'] = bikes_19['Id_Aparcamiento_Destino'].astype(int)
# split the previous data (needed for init state of bikes)
bikes_pre = bike_df_all[bike_df_all.Fecha_Devolucion.dt.year < 2019].reset_index()
bikes_pre['Id_Aparcamiento_Destino'] = bikes_pre['Id_Aparcamiento_Destino'].astype(int)
bikes_pre = bikes_pre[['Id_Tag_Bicicleta', 'Fecha_Devolucion', 'Id_Aparcamiento_Destino']].dropna()
# get all unique stations
unique_stations = pd.read_csv('raw_data/station_location_with_id.csv', sep=';').drop(columns=['photo', 'webDescription'])



In [5]:
# assign df a individual hour of the year per row
def get_hr(x):
    time = x
    hour = dt.datetime(time.year, time.month,time.day,time.hour)
    return hour
    
bikes_19.loc[:,'hour'] = bikes_19.Fecha_Devolucion.apply(get_hr)

In [6]:
# make sure df is sorted chronologically
bikes_19 = bikes_19.sort_values(by='Fecha_Devolucion')

# list all the appearing bikes and stations per individual hour of the year
bikes_19_by_hour = bikes_19.copy().groupby('hour').agg({'Id_Tag_Bicicleta':lambda x: list(x), 
                                                        'Id_Aparcamiento_Destino':lambda x: list(x),
                                                        'Id_Aparcamiento_Origen':lambda x: list(x)})

# compute the init state of all bikes prior to 2019 to get a starting balance of bikes
# we assume that the bike in its last position wasn't 'touched', thus remained in the station until the 02.01.2019
bikes_pre = bikes_pre.sort_values(by='Fecha_Devolucion', ascending=False) # walk backwards through df to find 'last' position of each bike
stations_init_df = bikes_pre.groupby('Id_Tag_Bicicleta').agg({'Id_Aparcamiento_Destino':'first'}) # first means last in this case

# count the bikes at the stations (and pad the non appearing ones)
stations_init_pre = Counter({i:0 for i in unique_stations.ID})
stations_init_pre.update(Counter(dict(stations_init_df.Id_Aparcamiento_Destino.value_counts())))


In [7]:
# only keep the _last_ entry of a bike per hour (dict will overwrite)
# we will count pick-up and drop-off in the SAME time, since the avg. renting duration is about 10 mins. 
bikes_19_by_hour['bikes_loc'] = None
for c, row in bikes_19_by_hour.iterrows():
#     print(c) # check progress
    dct = {}
    for i, bike in enumerate(row.Id_Tag_Bicicleta):
        dct[bike] = (row.Id_Aparcamiento_Destino[i], row.Id_Aparcamiento_Origen[i]) # create route dict for each bike in the hour
    bikes_19_by_hour.at[c, 'bikes_loc'] = dct

# count the pickups and drop offs for each station per hour
bikes_19_by_hour['station_movement'] = None
for c, row in bikes_19_by_hour.iterrows():
    stations = row.bikes_loc.values()
    orgs = [i[1] for i in stations] # unpack the rout tuples
    dests = [i[0] for i in stations]
    dct_org = Counter(orgs) # count how many interactions per station
    dct_dest = Counter(dests)
    dct_acum = {i:dct_dest.get(i,0)-dct_org.get(i,0) for i in set(dct_dest) | set(dct_org)} # subtract the pickups from the dropoffs
    bikes_19_by_hour.at[c, 'station_movement'] = dct_acum
           
# update the init station count by the movements
# stations_init_dict = stations_init_pre # copy dict
bikes_19_by_hour['station_count'] = None
for c, row in bikes_19_by_hour.iterrows():
    movement = Counter(row.station_movement)
    stations_init_pre.update(movement) # update moved items
    bikes_19_by_hour.at[c, 'station_count'] = dict(stations_init_pre)



# move hour to columns and only leave the dict
bikes_19_by_hour = bikes_19_by_hour[['station_movement', 'station_count']].reset_index()
        
# make dict column visible
bikes_19_by_hour.head(10).style.set_properties(subset=['station_count'], **{'width': '50'})


,hour,station_movement,station_count
0,2019-01-02 05:00:00,"{1: -2, 2: -1, 3: -1, 5: 1, 6: -1, 7: -5, 8: 0, 9: 1, 10: 3, 12: -1, 20: 0, 21: -1, 23: 1, 27: -1, 28: 1, 33: 1, 34: -1, 37: 1, 38: -1, 39: -1, 40: 1, 41: 1, 42: -1, 43: 1, 44: 1, 47: 0, 49: 1, 51: -1, 53: -1, 54: 3, 56: -2, 57: 1, 58: -1, 60: 1, 63: 2, 64: 2, 65: 1, 67: -1, 89: -1}","{1: 167, 2: 112, 3: 78, 4: 67, 5: 53, 6: 119, 7: 199, 8: 127, 9: 41, 10: 58, 11: 58, 12: 198, 13: 29, 19: 18, 20: 26, 21: 52, 22: 34, 23: 60, 24: 47, 27: 46, 28: 18, 32: 25, 33: 16, 34: 40, 35: 17, 36: 90, 37: 22, 38: 26, 39: 102, 40: 100, 41: 56, 42: 35, 43: 30, 44: 42, 45: 20, 46: 38, 47: 32, 48: 11, 49: 22, 51: 12, 53: 44, 54: 19, 55: 44, 56: 26, 57: 46, 58: 31, 59: 13, 60: 27, 61: 8, 62: 59, 63: 27, 64: 29, 65: 23, 66: 74, 67: 95, 70: 19, 89: 63, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 118: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 138: 0, 139: 0, 140: 0, 141: 0, 142: 0, 69: 10, 14: 4, 31: 4, 16: 1, 50: 1, 15: 1}"
1,2019-01-02 06:00:00,"{1: -20, 2: 0, 3: -1, 4: 2, 5: 0, 6: -2, 7: -4, 8: -12, 9: 4, 10: 1, 11: -2, 12: -12, 13: 0, 19: 4, 20: -2, 21: -5, 22: -3, 23: -9, 24: 4, 27: -7, 28: 7, 32: 1, 33: 7, 34: 5, 35: 2, 36: -1, 37: 1, 38: 2, 39: 3, 40: 0, 41: -1, 42: 3, 43: 5, 44: -1, 45: 1, 46: -5, 47: 3, 48: -4, 49: 2, 51: 2, 53: -3, 54: 7, 55: -1, 56: -1, 57: 12, 59: 5, 60: 1, 62: 2, 63: 12, 64: -2, 65: -1, 66: 1, 67: -1, 89: 1}","{1: 147, 2: 112, 3: 77, 4: 69, 5: 53, 6: 117, 7: 195, 8: 115, 9: 45, 10: 59, 11: 56, 12: 186, 13: 29, 19: 22, 20: 24, 21: 47, 22: 31, 23: 51, 24: 51, 27: 39, 28: 25, 32: 26, 33: 23, 34: 45, 35: 19, 36: 89, 37: 23, 38: 28, 39: 105, 40: 100, 41: 55, 42: 38, 43: 35, 44: 41, 45: 21, 46: 33, 47: 35, 48: 7, 49: 24, 51: 14, 53: 41, 54: 26, 55: 43, 56: 25, 57: 58, 58: 31, 59: 18, 60: 28, 61: 8, 62: 61, 63: 39, 64: 27, 65: 22, 66: 75, 67: 94, 70: 19, 89: 64, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 118: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 138: 0, 139: 0, 140: 0, 141: 0, 142: 0, 69: 10, 14: 4, 31: 4, 16: 1, 50: 1, 15: 1}"
2,2019-01-02 07:00:00,"{1: -6, 2: -11, 3: -1, 4: -2, 5: -5, 6: -7, 7: -1, 8: -18, 9: -6, 10: 3, 11: -6, 12: -12, 13: -7, 19: 16, 20: -2, 21: -1, 22: 1, 23: -10, 24: -3, 27: 1, 28: -1, 32: 1, 33: 10, 34: -2, 35: 1, 36: 7, 37: 1, 38: 0, 39: 1, 40: 4, 41: -5, 42: 0, 43: 4, 44: 0, 45: 0, 46: -1, 47: 8, 48: 1, 49: -3, 51: 1, 53: 0, 54: 8, 55: -3, 56: -8, 57: 10, 58: -1, 59: 1, 60: -1, 61: -1, 62: 1, 63: 6, 64: 5, 65: 2, 66: 23, 67: 7, 89: 1}","{1: 141, 2: 101, 3: 76, 4: 67, 5: 48, 6: 110, 7: 194, 8: 97, 9: 39, 10: 62, 11: 50, 12: 174, 13: 22, 19: 38, 20: 22, 21: 46, 22: 32, 23: 41, 24: 48, 27: 40, 28: 24, 32: 27, 33: 33, 34: 43, 35: 20, 36: 96, 37: 24, 38: 28, 39: 106, 40: 104, 41: 50, 42: 38, 43: 39, 44: 41, 45: 21, 46: 32, 47: 43, 48: 8, 49: 21, 51: 15, 53: 41, 54: 34, 55: 40, 56: 17, 57: 68, 58: 30, 59: 19, 60: 27, 61: 7, 62: 62, 63: 45, 64: 32, 65: 24, 66: 98, 67: 101, 70: 19, 89: 65, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 118: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 138: 0, 139: 0, 140: 0, 141: 0, 142: 0, 69: 10, 14: 4, 31: 4, 16: 1, 50: 1, 15: 1}"
3,2019-01-02 08:00:00,"{1: -8, 2: -4, 3: 2, 4: -3, 6: 0, 7: -8, 8: -4, 9: -1, 10: 0, 11: -4, 12: 1, 13: -4, 19: 2, 20: 6, 2

In [10]:
bikes_19_by_hour.to_csv('raw_data/stations_count_hr.csv')

In [214]:
# get all bike ids numbers of last hour in 2018 and their locations
# # filter by before 2019 and match it with all apearing bikes, take the .head(1) to have the location of the bike

# then take station count dict and update appearing stations

# all done